# Práctica 1

Gonzalo Monedero - Marcos Molina - Nicolás Salgado - Iñigo Vázquez

## Preparación del entorno

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as ker
import tensorflow as tf
import random
import time
import collections

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Quitar warnings feos
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Preprocesamiento de datos

### Creación del dataset


In [ ]:
iris_RAW = pd.read_csv('../Data/iris.data', sep=',')
iris_OG = iris_RAW.copy()
salidas = iris_RAW.pop("class")

## Creación entradas y salidas

In [ ]:
entradas = iris_RAW.to_numpy()
mm = MinMaxScaler()
entradas = mm.fit_transform(entradas)

# Categorizamos salidas
label_encoder = LabelEncoder()
salidas = label_encoder.fit_transform(salidas)


# Funciones algoritmo genético

### Variables a definir

In [ ]:
"""
ngenes: nneuronas de cada individuo. Sacado a traves del L4P1-EstudioRed.ipynb
NPOB: Número de individuos de la población 
NGEN: Número de generaciones (ciclo completo sobre todos los individuos) 
Pm: Probabilidad de mutación: Probabilidad de que un gen dado mute 
NRES: Cada cuántas generaciones se imprime un resumen de la evolución del proceso 
NSAMPLE: Cada qué número de generaciones se saca un muestreo de la población 
"""

ngenes = 4 # NO TOCAR
npob = 10 # NO TOCAR
ngeneraciones = 200 # TOCAR
pm = 0.3 # TOCAR
nres = 10 # TOCAR
nsample = 50 # TOCAR

### Crear inidividuo

Como bien se dice en el enunciado, un individuo es un MLP sin entrenar, con tantos genes como hemos calculado. En nuestro caso son 7 genes, y el tamaño de entrada son 4. De esta manera creamos un MLP básico.

In [ ]:
def crearMLP(ngenesMLP, tam_entrada,tipo,pesos):

    MLP = ker.Sequential() # Creamos el modelo vacío

    # Añadimos la primera capa (tecnicamente dos, la de entrada (con la variable input_dim) y la oculta)
    MLP.add(ker.layers.Dense(ngenesMLP, input_dim = tam_entrada, activation='relu'))

    # Añadimos la capa de salida
    MLP.add(ker.layers.Dense(1,activation='sigmoid')) 

    # Por último tenemos que configurar el modelo antes de entrenarlo con lo dicho en el enunciado
    # Optimizador: Adam
    # Función error: Mean Squared Error
    # Metricas = Binary accuracy porque nos da el número de precisión de la red
    MLP.compile(optimizer='adam',loss='mean_squared_error',metrics=["accuracy"])

    if(tipo == 1): # Le metemos pesos ya predeterminados
        MLP.set_weights(pesos)

    return MLP

### Crear población

Al igual que antes hemos creado un individuo en este caso crearemos una población. Una población es un conjunto de individuos, en nuestro caso entonces realizaremos un array de población

In [ ]:
def crearPoblacion(tamPoblacion,ngenesMLP, tam_entrada):
    poblacion = []
    for i in range (tamPoblacion):
        time.sleep(1)
        individuo = crearMLP(ngenesMLP,tam_entrada,0,0)
        poblacion.append(individuo)
    return poblacion

### Evaluar población

El calculo del fitness de la población se realiza mediante el evaluate del MLP. De esta manera, simplemente tendremos que iterar toda la población y devolver un array con el fitness de cada individuo en la posición de la población.

In [ ]:
def evaluarPoblacion(poblacion,entradas_test,esperadas_test):
    evalPobl = []
    for individuo in poblacion:
        metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
        evalPobl.append(metricas_totales*150)
    return evalPobl

### Evaluar individuo

Adicionalmente por si es requerido, crearemos una evaluacion para un unico individuo

In [ ]:
def evaluarIndividuo(individuo,entradas_test,esperadas_test):
    metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
    evalInd = metricas_totales*150
    return evalInd

### Selección de replicar individuo

Posteriormente necesitaremos un algoritmo de montecarlo, que nos permita elegir la probablididad de selección de el individuo. El método de montecarlo es generar un número aleatorio entre [0,1], si este número es menor que la probabilidad de selección, el individuo se replica, si no, no. En nuestro caso, recibimos una población y la recorremos, viendo cuales se cogen o no. De esta manera, devolvemos un de booleano donde indicamos si ese individuo se replica o no.

In [ ]:
def seleccion(individuo,poblacion,entradas_test,esperadas_test):
    fis_totales = evaluarPoblacion(poblacion,entradas_test,esperadas_test)
    fi = evaluarIndividuo(individuo,entradas_test,esperadas_test)
    nAleatorio = random.uniform(0, 1)
    ps = fi / np.amax(fis_totales)
    print(fis_totales)
    return nAleatorio < ps

### Mutar

Secuencialmente para todos los genes del padre decide si cada gen se copia fielmente en el hijo o se muta usando un Método de Montecarlo simple sobre el valor de Pm. Si muta, cámbialo por otro valor aleatorio en el rango (-1, 1). Prueba también otras opciones de mutación. 

In [ ]:
def mutar(individuo):
    pesos_antiguos = individuo.get_weights()
    pesos_entrada_oculta = pesos_antiguos[0]
    pesos_oculta_salida = pesos_antiguos[2]

    for i in range(len(pesos_entrada_oculta)):
        for j in range(len(pesos_entrada_oculta[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_entrada_oculta[i][j] = random.uniform(-1, 1)

    for i in range(len(pesos_oculta_salida)):
        for j in range(len(pesos_oculta_salida[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_oculta_salida[i][j] = random.uniform(-1, 1)
    
    aux1 = np.array([0., 0., 0., 0.], dtype='float32')
    aux2 = np.array([0.], dtype='float32')
    pesos_nuevos = [pesos_entrada_oculta,aux1,pesos_oculta_salida,aux2] # Los pesos con mutación

    nuevoMLP = crearMLP(ngenes,4,1,pesos_nuevos)

    return nuevoMLP

### Simulación

In [ ]:
def simular():
    
    poblacion = crearPoblacion(npob,ngenes, 4) # Población inicial

    for i in range(ngeneraciones):
        for individuo in poblacion: # Recorremos toda la población
            contador = 0
            seleccionado = seleccion(individuo,poblacion,entradas,salidas) # Vemos si le toca mutarse o no
            if(seleccionado): # Lo mutamos y matamos alguien aleatorio
                nuevoIndividuo = mutar(individuo) # Lo mutamos
                if(evaluarIndividuo(individuo,entradas,salidas) < evaluarIndividuo(nuevoIndividuo,entradas,salidas)):
                    muerto = random.randint(0, npob-1) # Nos cargamos un random
                    poblacion[muerto] = nuevoIndividuo
                    print(evaluarIndividuo(individuo,entradas,salidas), evaluarIndividuo(nuevoIndividuo,entradas,salidas), evaluarIndividuo(poblacion[muerto],entradas,salidas), muerto)
                
                if(i % nres == 0 and i % nsample != 0): # imprime un resumen de la evolución del proceso 
                    pobFit = evaluarPoblacion(poblacion,entradas,salidas)
                    valMejInd = np.amax(pobFit)
                    nInd = collections.Counter(pobFit)[valMejInd]
                    print("======================= RESUMEN =======================")
                    print("Nº generacion", str(i+1))
                    print("Mejor individuo tiene: ", str(valMejInd))
                    print("Hay ", str(nInd), "con el mejor fitness, lo cual es un ", str((nInd/npob)*100),"%")
                    print("=======================================================")
                
                if(i % nsample == 0): # muestreo de la población 
                    pobFit = evaluarPoblacion(poblacion,entradas,salidas)
                    valMejInd = np.amax(pobFit)
                    nInd = collections.Counter(pobFit)[valMejInd]

                    nindmuestreo = npob * 0.2 # 20%
                    muestreo = random.sample(poblacion, nindmuestreo)
                    print("======================= RESUMEN Y MUESTREO =======================")
                    print("Nº generacion", str(i+1))
                    print("Mejor individuo tiene: ", str(valMejInd))
                    print("Hay ", str(nInd), "con el mejor fitness, lo cual es un ", str((nInd/npob)*100),"%")
                    print("Muestro de ", str(nindmuestreo), "individuos")
                    for individuoMuestreo in muestreo:
                        contador += 1
                        print("Individuo ", str(contador), "tiene fitness de", str(evaluarIndividuo(individuoMuestreo,entradas,salidas)))
                    print("==================================================================")

simular()